# 채도 올리기 
## 1. PIL 이용한 채도 올리기 
##### 이 코드 실행 해 본 결과 밝기가 높아져서 사진이 깨지는 현상 발생 

In [14]:
from PIL import Image
import os

# 이미지가 있는 폴더 경로
folder_path = 'C:\\Users\\김현\\Final_Project\\joyful'

# 폴더의 모든 파일을 순회
for filename in os.listdir(folder_path):
    if filename.endswith(".jpg"): # 이미지 파일 확장자에 따라서 수정 필요
        # 이미지 불러오기
        img_path = os.path.join(folder_path, filename)
        try:
            img = Image.open(img_path)
            img = img.convert("RGB")

            # 채도를 증가시키기 위해 각 픽셀의 RGB 값을 조절
            data = np.array(img)
            r, g, b = data[:,:,0], data[:,:,1], data[:,:,2]
            r = r * 1.2  # 적당한 비율로 조절
            g = g * 1.2  # 적당한 비율로 조절
            b = b * 1.2  # 적당한 비율로 조절

            # 값의 범위를 0~255로 유지
            r = np.clip(r, 0, 255)
            g = np.clip(g, 0, 255)
            b = np.clip(b, 0, 255)

            # 새로운 이미지 데이터 생성
            new_data = np.stack([r, g, b], axis=2)
            new_img = Image.fromarray(new_data.astype('uint8'), 'RGB')

            # 이미지 저장 (새로운 이름으로 저장)
            base_filename, ext = os.path.splitext(filename)
            new_filename = f"{base_filename}_inc_sat{ext}"
            new_img_path = os.path.join(folder_path, new_filename)
            new_img.save(new_img_path)

        except IOError:
            print(f'Could not read image: {img_path}')
            continue


## - inc 단어 들어간 파일들 전부 삭제 (추가 수집된 파일 삭제)

In [18]:
 import os
import glob

folder_path = 'C:\\Users\\김현\\Final_Project\\joyful'

# 'inc'가 들어간 모든 파일의 리스트를 가져옵니다.
files = glob.glob(os.path.join(folder_path, '*inc*'))

# 리스트에 있는 모든 파일을 삭제합니다.
for file in files:
    try:
        os.remove(file)
    except OSError as e:
        print(f"Error: {file} : {e.strerror}")


## 2. scikit-image 사용

In [12]:
pip install scikit-image

     ---------------------------------------- 24.5/24.5 MB 9.8 MB/s eta 0:00:00
     -------------------------------------- 313.3/313.3 kB 9.5 MB/s eta 0:00:00
     -------------------------------------- 223.6/223.6 kB 6.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from skimage import io, color, img_as_ubyte
import os
import numpy as np

folder_path = 'C:\\Users\\김현\\Final_Project\\joyful'  # 이미지 파일이 저장된 폴더 경로를 입력하세요.

for img_filename in os.listdir(folder_path):  # 폴더 내의 모든 파일을 순회합니다.
    if not img_filename.lower().endswith(('.png', '.jpg', '.jpeg')):  # 이미지 파일만 처리합니다.
        continue

    img_path = os.path.join(folder_path, img_filename)  # 이미지 파일 경로를 구합니다.
    base_filename, ext = os.path.splitext(img_filename)

    try:
        img = io.imread(img_path)
    except IOError:
        print(f'Could not read image: {img_path}')
        continue  # 이미지 파일을 읽지 못했을 경우, 다음 이미지로 넘어갑니다.

    if len(img.shape) != 3 or img.shape[2] != 3:  # 이미지가 RGB가 아닌 경우
        if len(img.shape) == 2:  # 흑백 이미지인 경우
            img = color.gray2rgb(img)  # 흑백 이미지를 RGB로 변환
        else:
            print(f'Unsupported image format: {img_path}')
            continue

    img_hsv = color.rgb2hsv(img)
    img_hsv[:, :, 1] = img_hsv[:, :, 1] * 1.5  # 채도를 1.5배로 증가시킵니다.
    img_adjusted = color.hsv2rgb(img_hsv)
    img_adjusted = img_as_ubyte(img_adjusted)  # 이미지 데이터를 0~255 범위의 정수로 변환합니다.
    new_filename = f"{base_filename}_inc_sat{ext}"
    new_img_path = os.path.join(folder_path, new_filename)
    try:
        io.imsave(new_img_path, img_adjusted)
    except Exception as e:
        print(f'Could not save image: {new_img_path}. Error: {e}')
